### Импортируем нужные данные

In [1]:
import numpy as np
import pandas as pd
import os
import re

from sklearn.metrics import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import nltk
nltk.download('wordnet')
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Считаем данные

In [2]:
train = []
for line in open('full_train.txt', 'r', encoding = 'utf-8'):
    train.append(line.strip())
    
test = []
for line in open('full_test.txt', 'r',encoding = 'utf-8'):
    test.append(line.strip())
    
X = train + test
y = 2 * [1 if i < 12500 else 0 for i in range(25000)]
X,y = shuffle(X,y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

### Предобработка данных

In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

X_train = [clean_text(x) for x in X_train]
X_test = [clean_text(x) for x in X_test]

### Наивный Байесовский классификатор

In [56]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
X_train_counts = cv.fit_transform(X_train)
X_test_counts = cv.transform(X_test)

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, y_train)
predicted = clf.predict(X_test_counts)

In [27]:
acc = accuracy_score(predicted, y_test)
f1_score = f1_score(predicted, y_test)
print('Accuracy score: {0}'.format(acc))
print('F1-score: {0}'.format(f1_score))

Accuracy score: 0.859
F1-score: 0.8564057301921381


### Логистическая регрессия

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
X_train_counts = cv.fit_transform(X_train)
X_test_counts = cv.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train_counts, y_train)
predicted = clf.predict(X_test_counts)

C:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [ ]:
acc = accuracy_score(predicted, y_test)
f1_score = f1_score(predicted, y_test)
print('Accuracy score: {0}'.format(acc))
print('F1-score: {0}'.format(f1_score))

Accuracy score: 0.882
F1-score: 0.8824388947927737


### Метод опорных векторов (SVM)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_counts = cv.fit_transform(X_train)
X_test_counts = cv.transform(X_test)

In [12]:
from sklearn.svm import LinearSVC
clf = LinearSVC().fit(X_train_counts, y_train)
predicted = clf.predict(X_test_counts)

C:\Anaconda\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [13]:
acc = accuracy_score(predicted, y_test)
micro_f1 = f1_score(predicted, y_test)
print('Accuracy score: {0}'.format(acc))
print('F1-score = {0:1.4f}'.format(micro_f1))

Accuracy score: 0.8634666666666667
F1-score=0.8633
F1-score: <function f1_score at 0x00000240A7C2FD08>


### RNN

In [4]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [5]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)

In [6]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = y_train

In [10]:
embed_size = 256
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(96, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 31500 samples, validate on 3500 samples
Epoch 1/3
31500/31500 [==============================] - 752s 24ms/step - loss: 0.4322 - acc: 0.7984 - val_loss: 0.2818 - val_acc: 0.8797
Epoch 2/3
31500/31500 [==============================] - 806s 26ms/step - loss: 0.2501 - acc: 0.9010 - val_loss: 0.2749 - val_acc: 0.8877
Epoch 3/3
31500/31500 [==============================] - 806s 26ms/step - loss: 0.1926 - acc: 0.9258 - val_loss: 0.2832 - val_acc: 0.8863


In [11]:
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)

In [12]:
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred,y_test)))
print('Accuracy: {0}'.format(accuracy_score(y_pred, y_test)))

F1-score: 0.8883638511518015
Accuracy: 0.8866
